In [1]:
from collections import namedtuple
import os
import urllib.request as request
from zipfile import ZipFile

In [4]:
os.getcwd()
os.chdir("../")
os.getcwd()

'c:\\Projects\\CODEBASE\\DeepCNN_Classifier'

In [5]:
DataIngestionConfig = namedtuple("DataIngestionConfig",[
    "root_dir",
    "source_url",
    "local_data_file",
    "unzip_dir"
        ])

In [6]:
from deepClassifier.constants import *
from deepClassifier.utils import read_yaml,create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,
            config_filepath= CONFIG_FILE_PATH,
            parms_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.parms = read_yaml(parms_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)->DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
                root_dir=config.root_dir,
                source_url = config.source_url,
                local_data_file=config.local_data_file,
                unzip_dir = config.unzip_dir
                )
        return data_ingestion_config        

        

In [8]:
# creating component
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, header_info = request.urlretrieve(url=self.config.source_url,
                                                        filename=self.config.local_data_file
                                                        )

    def _get_updated_list_files(self,list_of_files):
        return [f for f in list_of_files if f.endswith(".jpg") and ("Cat" in f or "Dog" in f)]
    
    def _preprocess(self, zf:ZipFile, f:str, working_dir:str):
        target_file_path = os.path.join(working_dir, f)
        if not os.path.exists(target_file_path):
            zf.extract(f,working_dir)

        if os.path.getsize(target_file_path)==0:
            os.remove(target_file_path)

    def unzip_and_clean(self):
        with ZipFile(file=self.config.local_data_file, mode="r") as zf:
            list_of_files = zf.namelist()
            updated_list_of_files = self._get_updated_list_files(list_of_files)

            for f in updated_list_of_files:
                self._preprocess(zf, f, self.config.unzip_dir)

In [9]:
config = ConfigurationManager()
data_ingestion_config = config.get_data_ingestion_config()
data_ingestion = DataIngestion(config =data_ingestion_config)
data_ingestion.download_file()
data_ingestion.unzip_and_clean()

[2022-10-04 02:20:45,908:INFO:common]: yaml file: configs\config.yaml loaded successfully
[2022-10-04 02:20:45,912:INFO:common]: yaml file: params.yaml loaded successfully
[2022-10-04 02:20:45,914:INFO:common]: created directory at:artifacts
[2022-10-04 02:20:45,917:INFO:common]: created directory at:artifacts/data_ingestion
